In [1]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import types

In [4]:
import pandas as pd

In [2]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

24/03/04 09:30:58 WARN Utils: Your hostname, pop-os resolves to a loopback address: 127.0.1.1; using 192.168.1.128 instead (on interface wlp0s20f3)
24/03/04 09:30:58 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/04 09:30:59 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
print(f"1. My pyspark version is {spark.version}")

1. My pyspark version is 3.4.2


In [ ]:
df_pandas = pd.read_csv("fhv_tripdata_2019-10.csv.gz", compression="gzip")

In [ ]:
df_pandas.head()

In [ ]:
df_pandas.dtypes

In [ ]:
schema = types.StructType(
    [
        types.StructField("dispatching_base_num", types.StringType(), True),
        types.StructField("pickup_datetime", types.TimestampType(), True),
        types.StructField("dropoff_datetime", types.TimestampType(), True),
        types.StructField("PULocationID", types.IntegerType(), True),
        types.StructField("DOLocationID", types.IntegerType(), True),
        types.StructField("SR_Flag", types.StringType(), True),
        types.StructField("Affiliated_base_number", types.StringType(), True),

    ]
)

In [ ]:
df = spark.read \
    .option("header", "true") \
    .schema(schema) \
    .csv("/home/pedro/Downloads/fhv_tripdata_2019-10.csv.gz")
    

In [ ]:
df = df.repartition(6)
df.write.parquet("fhv/2019/10/")

**Q3**: How many taxi trips were there on October 15th?

In [ ]:
df.registerTempTable("fhv_2019_10")

In [5]:
from pyspark.sql import functions as F

In [ ]:
df \
    .withColumn('pickup_datetime', F.to_date(df.pickup_datetime)) \
    .filter("pickup_datetime = '2019-10-15'") \
    .count()

**Q4**: Longest trip for the month

In [6]:
df = spark.read.parquet("fhv/2019/10/")

In [8]:
df.createOrReplaceTempView("fhv_2019_10")

In [ ]:
df.columns

In [21]:
df \
    .withColumn('duration', df.dropoff_datetime.cast('long') - df.pickup_datetime.cast('long')) \
    .withColumn('pickup_date', F.to_date(df.pickup_datetime)) \
    .groupBy('pickup_date') \
        .max('duration') \
    .orderBy('max(duration)', ascending=False) \
    .limit(5) \
    .show()

+-----------+-------------+
|pickup_date|max(duration)|
+-----------+-------------+
| 2019-10-28|   2272149000|
| 2019-10-11|   2272149000|
| 2019-10-31|    315620787|
| 2019-10-01|    252460901|
| 2019-10-17|     31658400|
+-----------+-------------+



In [23]:
spark.sql("""
SELECT
    to_date(pickup_datetime) AS pickup_date,
    MAX((CAST(dropoff_datetime AS LONG) - CAST(pickup_datetime AS LONG)) / 60/60) AS duration
FROM 
    fhv_2019_10
GROUP BY
    1
ORDER BY
    2 DESC
LIMIT 10;
""").show()

+-----------+------------------+
|pickup_date|          duration|
+-----------+------------------+
| 2019-10-28|          631152.5|
| 2019-10-11|          631152.5|
| 2019-10-31| 87672.44083333334|
| 2019-10-01| 70128.02805555557|
| 2019-10-17|            8794.0|
| 2019-10-26| 8785.166666666666|
| 2019-10-30|1464.5344444444445|
| 2019-10-25|1057.8266666666666|
| 2019-10-02| 770.2313888888889|
| 2019-10-23| 746.6166666666667|
+-----------+------------------+



**Q6**: Least used pickup zone?

In [36]:
df_zones = (
    spark.read.option("header", "true").csv("taxi+_zone_lookup.csv")
)
df_zones.createOrReplaceTempView("zones")

In [40]:
df_zones.printSchema()

root
 |-- LocationID: string (nullable = true)
 |-- Borough: string (nullable = true)
 |-- Zone: string (nullable = true)
 |-- service_zone: string (nullable = true)



In [42]:
spark.sql(
    """
SELECT
    pul.Zone,
    COUNT(1)
FROM 
    fhv_2019_10 fhv LEFT JOIN zones pul ON fhv.PULocationID = pul.LocationID
                      LEFT JOIN zones dol ON fhv.DOLocationID = dol.LocationID
GROUP BY 
    1
ORDER BY
    2 asc
LIMIT 5;
"""
).show()

+--------------------+--------+
|                Zone|count(1)|
+--------------------+--------+
|         Jamaica Bay|       1|
|Governor's Island...|       2|
| Green-Wood Cemetery|       5|
|       Broad Channel|       8|
|     Highbridge Park|      14|
+--------------------+--------+

